In [ ]:
import ray
import pandas as pd
import time 
import numpy as np
import binascii
import redis

In [ ]:
IDENTIFIER_LENGTH = 20

# This prefix must match the value defined in ray_redis_module.cc.
DB_CLIENT_PREFIX = b"CL:"


def hex_identifier(identifier):
    return binascii.hexlify(identifier).decode()


def identifier(hex_identifier):
    return binascii.unhexlify(hex_identifier)

def key_to_hex_identifiers(key):
    # Extract worker_id and task_id from key of the form
    # prefix:worker_id:task_id.
    offset = key.index(b":") + 1
    worker_id = hex_identifier(key[offset:(offset + IDENTIFIER_LENGTH)])
    offset += IDENTIFIER_LENGTH + 1
    task_id = hex_identifier(key[offset:(offset + IDENTIFIER_LENGTH)])
    return worker_id, task_id

import re
def clean(sometext):
    sometext = sometext.decode('UTF-8')
    ansi_escape = re.compile(r'\x1b[^m]*m')
    return ansi_escape.sub('', sometext)

In [ ]:
ray.init(num_cpus=3)

In [ ]:
# Function definitions...

@ray.remote
def example(x):
    time.sleep(np.random.random())
    return np.random.randn()

@ray.remote
class TestCls():
    def __init__(self):
        self.g = 1
        
    def to_go(self, x):
        return x

    
@ray.remote
class Outer():
    def __init__(self):
        self.f = 1
        self.test = TestCls.remote()
    
    def to_go2(self, x):
        return x * 2
    
    def error(self):
        return 1/0

In [ ]:
# Here, we generate data in redis for remote tasks
results = ray.get([example.remote(x) for x in range(4)])

# Generating data for Actor tasks
actor = TestCls.remote()
actor_results = ray.get([actor.to_go.remote(1)])

In [ ]:
global_state = ray.worker.global_state # Used to access redis client, but also has nice interface for certain information 
gworker = ray.worker.global_worker # Unused for now
rc_non = global_state.redis_client # Redis Client for interacting with redis without decoding
addr, port = gworker.redis_address.split(":")
rc = redis.StrictRedis(host=addr, port=port, decode_responses=True)

Key things included in redis from `rc.keys()`:
 - Event log
 - worker info
 - Remote functions
 - "Functions to run"
 - Drivers
 - Redis clients
 - Actor classes
 - Actors

In [ ]:
import pprint
pp = pprint.PrettyPrinter() # for printing dicts and lists in a manner easy for the eyes


for k in rc.keys():
    try:
        print(k)
        pp.pprint(rc.hgetall(k))
#         rc.hgetall(k) 
    except Exception:
        print(k, "Failed ")
# Certain return values, such as `event_log:*`, can only be accessed via list calls to redis

#         try:
#             print('#' * 10)
#             print(k)
#             pp.pprint(rc.lrange(k, 0, -1))
#         except Exception:
#             print(k, "Failed ")
            

## Actor Info

In [ ]:
# import unicode 
actor_list = []
for a_key in rc.keys("Actor:*"):
    v = rc.hgetall(a_key)
    v['actor_id'] = a_key
    v['class_id'] = hex_identifier(v['class_id'])
    actor_list.append(v)

actor_df = pd.DataFrame(actor_list)

# TODO: Fix actor_id

In [ ]:
actor_df

In [ ]:
actor_classes = []
for a_key in rc.keys("ActorClass:*"):
    
    v = rc.hgetall(a_key)
    del v['class'] # removed pickle hex for readability
    v['driver_id'] = hex_identifier(v['driver_id'])
    class_id = a_key.split(b':')[1]
    v['class_id'] = hex_identifier(class_id)
    actor_classes.append(v)
    
actor_class_df = pd.DataFrame(actor_classes)

In [ ]:
actor_class_df

## Function Info

## Node Info

In [ ]:
global_state.client_table()

In [ ]:
global_state.function_table()

In [ ]:
object_dict = {oid.hex(): v for oid, v in global_state.object_table().items()}
object_df = pd.DataFrame(object_dict)

# Tasks

In [ ]:
tt = global_state.task_table()
tt_list = list(tt.values())

for d in tt_list:
    d['TaskSpec']['ReturnObjectIDs'] = [oid.hex() for oid in d['TaskSpec']['ReturnObjectIDs']]

In [ ]:
from pandas.io.json import json_normalize

result = json_normalize(tt_list)

result

We can also parse the event logs in order to get fine grained timing for remote tasks. However, as a user, I'd probably only care about time taken in running the task -- this can be much refined.

In [ ]:
import json 
event_list = []

# Get and decode all task timing/event logs
for key in rc.keys("event_log*"):
    content = rc.lrange(key, 0, -1)
#     event_list.append(json.loads(content[0].decode())) 
    event_list.append(json.loads(content[0]))
    
# event_list

In [ ]:
from collections import defaultdict

# This seems to be the best way to do the event_log -> dataframe pipeline. 
# First generate a (key, [value]) mapping for all tasks and then apply some pandas operations to convert.

# event_dict is used to store timing info
event_dict = defaultdict(lambda: np.full(len(event_list), np.nan))

# info_dict is used to store meta data - such as function names and task id
info_dict = defaultdict(lambda: [None] * len(event_list))

for i, task_event in enumerate(event_list):
    for event in (task_event):
        time, label, startstop, info = event
        event_dict[(label, startstop)][i] = time
        if info:
            for k in info:
                info_dict[k][i] = info[k]

In [ ]:
# The tuple keys for `event_dict` produce a hierarchical index, which could be useful. However, joining or merging it 
# with other non-hierarchical indices will throw away this structure.

edf = pd.DataFrame(event_dict) 
edf.rename(columns={1: 'start', 2:'end'}, inplace=True)
edf

In [ ]:
idf = pd.DataFrame(info_dict)
# idf.columns = pd.MultiIndex.from_tuples([(c, '') for c in idf]) # this is non-idempotent!
idf

In [ ]:
remote_table = pd.concat([idf, edf], axis=1)

In [ ]:
# Here, we join two tables together as an example
idf.merge(result, left_on="task_id", right_on="TaskSpec.TaskID")

In [ ]:
# Example to get Error, function_name, IP address, Actor ID, ParentID 

# TODO:

- Error messages logging (specifics - which node, which function call, which actor, what time)
  - Getting Error messages from Redis is redundant because we already get info in the event_log. Error messages provide extra information such as `error_id` and `type`, which don't seem particularly useful.
  - Create table for workers (Id, socket info, node IP address)
  - Get multinode setting - test out client table
  - Write out an example for error tracing